In [14]:
import pymongo
from dotenv import load_dotenv
import os
import itertools
import haversine as hs
import json
import time
from math import pi, sqrt, cos, prod

load_dotenv()

mongodb_uri = os.environ.get("MONGODB_URI")
client = pymongo.MongoClient(mongodb_uri)
db = client["schedule"]
classes = db["classes"]
locations = db["locations"]


In [15]:
weights = {
    "distance": 40,
    "back_to_back": 10,
    "time": 50,
    "section": 30,
    "lunch": 20
}

class ScheduleException(Exception):
    pass

class ScheduleOverlapException(Exception):
    # When a schedule is not possible because 2 classes overlap time-wise
    pass

In [17]:
def check_for_no_section(classes_to_take, open_sections_filter_applied):
    for clas in classes_to_take:
        if len(clas["sections"]) == 0:
            if (open_sections_filter_applied):
                raise ScheduleException(f"No open sections found for {clas['code']} {clas['number']} in your specified time range. Please try again with a different time range.")
            else:
                raise ScheduleException(f"No sections found for {clas['code']} {clas['number']} in your specified time range. Please try again with a different time range.")


def fetch_section_data(class_list):
    """
    Fetches the section data from the database
    """
    for class_ in class_list:
        clas_mongo_object = classes.find_one({"code": class_["code"], "number": class_[
                                             "number"], "year": 2024, "semester": "spring"})
        class_["sections"] = []
        for section in clas_mongo_object["sections"]:
            if section["crn"] in class_["crn_list"]:
                section["class"] = {
                    "code": class_["code"],
                    "number": class_["number"]
                }
                # Drop online lecture sections
                meeting_type_codes = [meeting["type_code"] for meeting in section["meetings"]]
                if ("OLC" in meeting_type_codes):
                    continue
                class_["sections"].append(section)

    return class_list


def is_preferred_time(section, start_time, end_time):
    earliest_meeting = start_time
    latest_meeting = end_time

    for meeting in section["meetings"]:
        if meeting['start_time'] is None or meeting['end_time'] is None:
            continue
        if meeting['start_time'].hour < earliest_meeting:
            earliest_meeting = meeting['start_time'].hour
        if meeting['end_time'].hour > latest_meeting:
            latest_meeting = meeting['end_time'].hour

    return earliest_meeting >= start_time and latest_meeting <= end_time


def delete_unpreferred_sections(class_list, start_time, end_time):
    for clas in class_list:
        clas["sections"] = [section for section in clas['sections'] if is_preferred_time(section, start_time, end_time)]
    return class_list


def delete_closed_sections(class_list, open_sections_only):
    if open_sections_only:
        for clas in class_list:
            clas["sections"] = [section for section in clas['sections']
                                if section['enrollment_status'].lower() != 'closed']
    return class_list


def apply_hard_filters(user_preferences):
    class_list = fetch_section_data(user_preferences["classes"])
    class_list = delete_unpreferred_sections(class_list, user_preferences["start_time"], user_preferences["end_time"])
    check_for_no_section(class_list, False)  # Check if any class has no sections

    class_list = delete_closed_sections(class_list, user_preferences["open_sections_only"])

    check_for_no_section(class_list, True)  # Check if any class has no sections

    return class_list


def remove_duplicates(grouped_sections, pref_sections):
    hash_list = set()
    new_list = []
    for section in grouped_sections:
        section_hash = f"{section['meetings'][0]['coordinates']}_{section['meetings'][0]['start_time']}_{section['meetings'][0]['end_time']}_{section['meetings'][0]['days']}"
        if section_hash not in hash_list:
            hash_list.add(section_hash)
            new_list.append(section)
        elif section['crn'] in pref_sections:
            new_list.append(section)

    return new_list


def generate_schedule_combinations(class_list, user_preferences):
    """
    Generates all possible schedules
    """
    groups = {}

    # Split classes into groups based on meeting types
    for clas in class_list:
        base_group_name = f"{clas['code']}_{clas['number']}_"
        for section in clas['sections']:
            list_of_meeting_types = [meeting['type_code']
                                     for meeting in section['meetings']]
            group_key = base_group_name + \
                "_".join([str(x) for x in list_of_meeting_types])
            groups[group_key] = groups.get(group_key, []) + [section]

    un_duplicated_groups = {}


    # Remove groups with section which have the same days, times, and locations
    for key, value in groups.items():
        un_duplicated_groups[key] = remove_duplicates(value, user_preferences["pref_sections"])

    possible_schedules_len = prod([len(x) for x in un_duplicated_groups.values()])
    possible_schedules = itertools.product(*un_duplicated_groups.values())
    print(f"Possible schedules: {possible_schedules_len}")
    return possible_schedules

In [18]:
# 0.3407275676727295, 0.31998157501220703


def pythagorian_distance(coords1, coords2):
    lat1 = coords1[0]  
    lon1 = coords1[1]  
    lat2 = coords2[0] 
    lon2 = coords1[1] 

    avgLatDeg = (lat1 + lat2) / 2
    avgLat = avgLatDeg * (pi/180)

    d_ew = (lon2 - lon1) * cos(avgLat)
    d_ns = (lat2 - lat1)
    approxDegreesSq = (d_ew * d_ew) + (d_ns * d_ns)
    d_degrees = sqrt(approxDegreesSq)
    EarthAvgMeterPerGreatCircleDegree = (6371000 * 2 * pi) / 360
    return d_degrees * EarthAvgMeterPerGreatCircleDegree


def list_of_successive_distances(schedule):

    list_of_distances = []
    for day in schedule:
        for hour in range(0, 23):
            if schedule[day][hour] is not None and schedule[day][hour + 1] is not None:
                coords1 = schedule[day][hour]['meetings'][0]['coordinates']
                coords2 = schedule[day][hour + 1]['meetings'][0]['coordinates']
                if coords1 is not None and coords2 is not None:
                    list_of_distances.append(pythagorian_distance(coords1, coords2))

    return [x for x in list_of_distances if x != 0.0]


def distance_score(schedule, max_distance) -> float:
    # Returns a score between 0 and distance_weight based on how far apart the classes are
    list_of_distances = list_of_successive_distances(schedule)

    if len(list_of_distances) == 0:
        return 0

    mean_distance = sum(list_of_distances) / len(list_of_distances)

    # Find the distance score
    if mean_distance > max_distance:
        percent_over = (mean_distance - max_distance) / max_distance
        return (1 - percent_over) * weights["distance"]
    elif mean_distance <= max_distance:
        return weights["distance"]

In [19]:

def back_to_back_score(schedule, back_to_back) -> float:
    # Returns a score between 0 and back_to_back_weight based on how many classes are back to back
    back_to_back_count = 0
    total_count = 0

    for day in schedule:
        for hour in range(0, 23):
            if schedule[day][hour] is not None:
                total_count += 1
                if schedule[day][hour + 1] is not None and schedule[day][hour]['crn'] != schedule[day][hour + 1]['crn']:
                    back_to_back_count += 1

    if total_count == 0:
        return 0
    elif back_to_back:
        return (back_to_back_count / total_count) * weights["back_to_back"]
    else:
        return (1 - (back_to_back_count / total_count)) * weights["back_to_back"]


In [20]:
# 0.323688268661499, 
def time_score(schedule, pref_time):
    # Returns a score between 0 and time_weight based on how close the classes are to the preferred time
    time_list = []

    for day in schedule:
        for hour in range(0, 23):
            if schedule[day][hour] is not None:
                time_list.append(hour)

    if len(time_list) == 0:
        return 0
    else:
        mean_time = sum(time_list) / len(time_list)
        return (1 - (abs(mean_time - pref_time) / pref_time)) * weights["time"]


In [21]:
def section_score(list_of_sections, pref_sections) -> int :
    # section score from 0 to weight
    # Computes percentage of preferred sections
    crn_list = [x['crn'] for x in list_of_sections]
    fraction_of_pref_sections = len(set(crn_list).intersection(set(pref_sections))) / len(pref_sections)
    return fraction_of_pref_sections * weights["section"]

In [22]:
def lunch_score(schedule, lunch):
    # lunch score from 0 to weight
    lunch_count = 0
    for day in schedule:
        for hour in range(0, 23):
            if schedule[day][hour] is None and hour >= lunch['start'] and hour < lunch['end']:
                duration_hours = [schedule[day][x] is None for x in range(hour, hour + lunch['duration']) if x < 23]
                if all(duration_hours):
                    lunch_count += 1
                    break
    return ((lunch_count)/5) * weights["lunch"]

In [23]:
# todo : try with time class and custom adder method

In [24]:
def hash_schedules(class_list) :
    hashmap = {}

    for clas in class_list:
        for section in clas['sections']:
            for meeting in section['meetings']:
                if meeting['start_time'] is None or meeting['end_time'] is None:
                    continue
                start_hour = meeting['start_time'].hour
                end_hour = meeting['end_time'].hour
                section_dict = {}
                for day in list(meeting['days'].strip()):
                    section_dict[day] = (start_hour, end_hour)
                hashmap[section['crn']] = section_dict
    
    return hashmap

In [25]:
def has_schedule_overlap(list_of_sections, schedule_time_hashmap) :
    days = {}
    for section in list_of_sections:
        for day in schedule_time_hashmap[section['crn']] :
            days[day] = days.get(day, []) + [schedule_time_hashmap[section['crn']][day]]

In [26]:
# def convert_to_time_based(list_of_sections, schedule_time_hashmap):
#     schedule_slots = {
#         'W': [None]*23,
#         'R': [None]*23,
#         'F': [None]*23,
#         'T': [None]*23,
#         'M': [None]*23,
#     }

#     for section in list_of_sections:
        


In [27]:
def compute_schedule_score(list_of_sections, user_preferences, schedule_time_hashmap) -> float:
    
    schedule = convert_to_time_based(list_of_sections, schedule_time_hashmap)
    
    if user_preferences['max_distance'] > 10_000:
        ds = 0
    else :
        ds = distance_score(schedule, user_preferences["max_distance"])

    btb = back_to_back_score(schedule, user_preferences["back_to_back"])
    ts = time_score(schedule, user_preferences["pref_time"])
    ss = section_score(schedule, user_preferences["pref_sections"])
    ls = lunch_score(schedule, user_preferences["lunch"])
    return (ds + btb + ts + ss + ls), schedule

In [28]:
def sort_schedules(possible_schedules, user_preferences, class_list):
    sorted_schedules = []
    schedule_time_hashmap = hash_schedules(class_list)

    for schedule in possible_schedules:
        try :
            score, time_schedule = compute_schedule_score(schedule, user_preferences, schedule_time_hashmap)
            sorted_schedules.append(
                {
                    "time_schedule": time_schedule,
                    "schedule": schedule,
                    "score": score
                }
            )
        except ScheduleOverlapException:
            continue
    if len(sorted_schedules) == 0:
        raise ScheduleException("No possible schedules found. Please try again with different preferences.")
    sorted_schedules.sort(key=lambda x: x["score"], reverse=True)
    return sorted_schedules

In [29]:
def get_schedule(user_preferences):
    """
    Returns the schedule for the user
    """
    class_list = apply_hard_filters(user_preferences)
    print('Applied hard filters')
    possible_schedules = generate_schedule_combinations(class_list, user_preferences)

    schedule_2 = [list(x) for x in possible_schedules]
    print('hi', schedule_2[0], type(schedule_2[0]))
    with open('possible_schedules.json', 'w') as f:
        json.dump(schedule_2, f, indent=4)
        
    print('Generated schedule combinations')
    possible_schedules = sort_schedules(possible_schedules, user_preferences, class_list)
    print('Sorted schedules')
    print(possible_schedules[0])

In [30]:
user_prefs = {"classes": [{"code": "ECE", "number": "110", "crn_list": ["32463", "32460", "52912", "32470", "52914", "52910", "32466", "32461", "52913", "32456", "52911", "63640", "32459", "32464", "32471", "52909", "61723", "57693"]}, {"code": "ECE", "number": "220", "crn_list": ["61635", "61638", "61640", "61641", "61642", "61643", "64369", "61629", "63649", "63650"]}, {"code": "PHYS", "number": "212", "crn_list": ["38029", "38032", "57914", "56034", "58663", "51211", "58664", "53105", "38051", "38057", "38104", "38059", "58662", "57937", "38062",
                                                                                                                                                                                                                                                                                                                                                                                                                                                                        "57963", "58985", "58987", "58988", "58978", "58979"]}, {"code": "MATH", "number": "241", "crn_list": ["50318", "46053", "46054", "46056", "46057", "46058", "48355", "46061", "46062", "46065", "46066", "46063", "46064", "59561", "58071", "46060", "46067", "46059", "52979", "46070", "46071", "46072", "46073", "47543", "67808", "50322", "55923", "55924", "55925", "55922", "55926", "56027", "56026", "55921", "46074"]}, {"code": "CWL", "number": "207", "crn_list": ["65011"]}], "pref_sections": ["52914", "63640", "32471", "61643", "61642", "38096", "38088", "50318"], "classes_1": ["ECE 110", "ECE 220", "PHYS 212", "MATH 241", "CWL 207"], "open_sections_only": False, "start_time": 10, "end_time": 21, "pref_time": 16, "lunch": {"start": 11, "end": 14, "duration": 1}, "max_distance": 800, "back_to_back": True}
test_outputs = get_schedule(user_prefs)

Applied hard filters
Possible schedules: 4976640
hi [{'crn': '32470', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2024/spring/ECE/110/32470.xml', 'section_number': 'AB4', 'status_code': 'A', 'part_of_term': '1', 'section_status_code': 'A', 'enrollment_status': 'Closed', 'section_text': None, 'start_date': datetime.datetime(2024, 1, 16, 0, 0), 'end_date': datetime.datetime(2024, 5, 1, 0, 0), 'meetings': [{'id': '0', 'type': 'Laboratory', 'type_code': 'LAB', 'start_time': datetime.datetime(1900, 1, 1, 12, 0), 'end_time': datetime.datetime(1900, 1, 1, 14, 50), 'days': 'M', 'room_number': '1001', 'building_name': 'Electrical & Computer Eng Bldg', 'coordinates': [40.115077164725506, -88.22815195075593], 'instructors': []}], 'class': {'code': 'ECE', 'number': '110'}}, {'crn': '32464', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2024/spring/ECE/110/32464.xml', 'section_number': 'AL1', 'status_code': 'A', 'part_of_term': '1', 'section_status_code':

TypeError: Object of type datetime is not JSON serializable